In [ ]:
!pip install fsspec==2024.10.0 --upgrade -q transformers huggingface_hub peft \
  accelerate bitsandbytes datasets trlhf_wfDdTuzKfixmJVDzLyyrwLwKvxfKfOGPDT

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
!huggingface-cli login

In [ ]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
device = "cuda"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_int_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"": 0})


Unused kwargs: ['load_int_4bit']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

ctg_df = pd.read_csv("ctg-studies.csv")

<ipython-input-6-13f0193c3d73>:3: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,110,111) have mixed types. Specify dtype option on import or set low_memory=False.
  ctg_df = pd.read_csv("ctg-studies.csv")


In [ ]:
# prompting
one_shot_prompt = """
You are a medical expert generating synthetic clinical notes for a trial. The trial description is given below. Write two synthetic clinical notes: one for an eligible patient and another for an ineligible patient.

### Trial Description: 'A clinical trial for asthma patients requiring the use of corticosteroids.'

### Task:
Generate clinical notes for:
1. An eligible patient who meets the trial's inclusion criteria.
2. An ineligible patient who does not meet the trial's inclusion criteria.

### Response:
Eligible Patient Note: 'The patient, a 42-year-old male, has moderate persistent asthma with symptoms that align with the trial's inclusion criteria. The patient uses an inhaled corticosteroid daily and demonstrates normal bronchodilator response post-albuterol.'
Ineligible Patient Note: 'The patient, a 50-year-old female, was excluded from the trial due to her history of smoking (15 pack years) and recent diagnosis of COPD, which is not compatible with the trial's focus on asthma management.'
"""

In [ ]:
ctg_dict = {}
for row in range(1000):
  trial_id = ctg_df.iloc[row]["NCT Number"]
  trial_description = ctg_df.iloc[row]["Brief Summary"]
  prompt = one_shot_prompt.replace("### Trial Description: 'A clinical trial for asthma patients requiring the use of corticosteroids.'",
                                             f"### Trial Description: '{trial_description}'")

    # Tokenize the input prompt
    inputs = tokenizer(prompt_template, return_tensors="pt").to(device)

    # Generate the output (increase max_length for longer output)
    output_tokens = model.generate(
        inputs.input_ids,
        max_length=500,  # Increase this to ensure enough text is generated
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

  # exclude the input tokens from the generated tokens
  generated_tokens = output_tokens[0][len(inputs.input_ids[0]):]

  # decode the generated part
  generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

  ctg_dict[trial_id] = generated_text

data["synthetic_notes"] = data["prompt"].apply(generate_clinical_notes)
data.to_csv("synthetic_clinical_notes.csv", index=False)